In [ ]:
from combinatorial_gwas.phenotypes import quality_heritability_phenos

/lab/corradin_biobank/FOR_AN/combinatorial_GWAS/.venv/lib/python3.8/site-packages/kedro/extras/datasets/pandas/csv_dataset.py:158: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return pd.read_csv(fs_file, **self._load_args)


In [ ]:
quality_heritability_phenos.head() #27 significantly heritable phenotypes

In [ ]:
display_cols = ['description', 'h2_liability', 'h2_sig', 'confidence', 'n_cases', 'n_controls', 'prevalence']


In [ ]:
!bgadd /combigwas

In [ ]:
!bgmod -L 30 /combigwas

In [ ]:
from functools import partial

get_command = partial("bsub -g /combigwas -q corradin -u '{email}' -J {pheno}_{threshold}_{max_samples} /lab/corradin_biobank/FOR_AN/combinatorial_GWAS/.venv/bin/python  /lab/corradin_biobank/FOR_AN/combinatorial_GWAS/notebooks/neural_network_training/nn.py {pheno} --network {nn} --thresholds {threshold} --create-test-output -m {max_samples} -r 42".format)

In [ ]:
email = "anhoang@wi.mit.edu"

## Check that the command is correct

In [ ]:
get_command(pheno = "I83", threshold = 1e-5, max_samples = 100_000, email = email)

"bsub -q corradin -u 'anhoang@wi.mit.edu' -J I83_1e-05_100000 /lab/corradin_biobank/FOR_AN/combinatorial_GWAS/.venv/bin/python  /lab/corradin_biobank/FOR_AN/combinatorial_GWAS/notebooks/neural_network_training/nn.py I83 --thresholds 1e-05 --create-test-output -m 100000 "

## Execute the command

In [ ]:
!{get_command(pheno = "I83", threshold = 1e-5, max_samples = 100_000)}

Job <9642219> is submitted to queue <corradin>.


## Execute in a loop

In [ ]:
for pheno in quality_heritability_phenos.sort_values("h2_liability", ascending=False).index:
    for threshold in [1e-6, 1e-7, 1e-8]:
        for max_samples in [1000, 10_000, 100_000, 300_000, 500_000]:
            command = get_command(pheno = pheno, threshold = threshold, max_samples = max_samples, email = email)
            !{command}

Job <9642232> is submitted to queue <corradin>.
Job <9642233> is submitted to queue <corradin>.
Job <9642234> is submitted to queue <corradin>.


In [ ]:
email = 'premc@mit.edu'
for pheno in ['I83']:
    for threshold in [1e-8]:
        for max_samples in [1000, 10_000, 100_000, 300_000, 500_000]:
            command = get_command(pheno = pheno, threshold = threshold, max_samples = max_samples, email = email, nn='dnn')
            !{command}

Job <9970684> is submitted to queue <corradin>.
Job <9970685> is submitted to queue <corradin>.
Job <9970686> is submitted to queue <corradin>.
Job <9970688> is submitted to queue <corradin>.
Job <9970689> is submitted to queue <corradin>.


In [ ]:
# email = 'premc@mit.edu'
# for pheno in ['I83', 'I25', 'G56', 'N81', 'M16', 'M17', 'I48', 'C44', 'K80', 'I21']:
#     for network in ['dnn', 'cnn', 'deeper-cnn', 'deeper-dnn']:
#         for threshold in [1e-6, 1e-7, 1e-8]:
#             for max_samples in [1000, 10_000, 100_000, 300_000, 500_000]:
#                 command = get_command(pheno = pheno, threshold = threshold, max_samples = max_samples, email = email, nn=network)
#                 !{command}

## Check the status of jobs

In [ ]:
!bjobs -w -g /combigwas

JOBID   USER    STAT  QUEUE      FROM_HOST   EXEC_HOST   JOB_NAME   SUBMIT_TIME
1529867 anhoang RUN   corradin   thyroxine   c4b7        I83_1e-08_1000 May 11 23:44


In [ ]:
#!bpeek 205135

In [ ]:
# jobs = [475127, 475128, 475129, 475130, 475131, 475132, 475133, 475134, 475135, 475136, 475137, 475138, 475139, 475140, 475141, 475143, 475146, 475150, 475153, 475156, 475157, 475158, 475159, 475160, 475161, 475162, 475163, 475164, 475165, 475166]
# kills = [f'bkill {job}' for job in jobs]
# for command in kills:
# #     !{command}
#     pass

In [ ]:
!bgadd /combigwas-interp

Job group </combigwas-interp> was added.


In [ ]:
!bgmod -L 30 /combigwas-interp

Job group /combigwas-interp is modified.


In [ ]:
# email = 'premc@mit.edu'
# for pheno in ['I83', 'I25', 'G56', 'N81', 'M16', 'M17', 'I48', 'C44', 'K80', 'I21']:
#     for network in ['dnn', 'cnn', 'deeper-cnn', 'deeper-dnn']:
#         for threshold in [1e-6, 1e-7, 1e-8]:
#             for max_samples in [1000, 10_000, 100_000, 300_000, 500_000]:
#                 command = get_command(pheno = pheno, threshold = threshold, max_samples = max_samples, email = email, nn=network)
#                 !{command}

# get_command = partial("bsub -g /combigwas -q corradin -u '{email}' -J {pheno}_{threshold}_{max_samples} /lab/corradin_biobank/FOR_AN/combinatorial_GWAS/.venv/bin/python  /lab/corradin_biobank/FOR_AN/combinatorial_GWAS/notebooks/neural_network_training/nn.py {pheno} --network {nn} --thresholds {threshold} --create-test-output -m {max_samples} -r 42".format)


def join(ls, sep='_'):
    return sep.join(str(l) for l in ls)

def get_name(networks, dates, phenotypes, thresholds, max_samples):
    return f"networks__{join(networks)}__dates__{join(dates)}__phenotypes__{join(phenotypes)}__thresholds__{join(thresholds)}__max_samples__{join(max_samples)}"

interp = lambda email, networks, dates, pheno, thresholds, max_samples: f"bsub -g /combigwas-interp -q corradin -u '{email}' -J interp_{get_name(networks, dates, pheno, thresholds, max_samples)} /lab/corradin_biobank/FOR_AN/combinatorial_GWAS/.venv/bin/python /lab/corradin_biobank/FOR_AN/combinatorial_GWAS/notebooks/neural_network_training/interp.py -n {join(networks, ' ')} -d {join(dates, ' ')} -p {join(pheno, ' ')} -t {join(thresholds, ' ')} -m {join(max_samples, ' ')}"

In [ ]:
print(interp(email, ['dnn', 'cnn'], ['4_29_2021'], ['I83', 'I25', 'G56', 'N81', 'M16', 'M17', 'I48', 'C44', 'K80', 'I21'], [6, 7, 8], [1000, 10_000, 100_000, 300_000, 500_000]))

bsub -g /combigwas-interp -q corradin -u 'premc@mit.edu' -J interp_networks__dnn_cnn__dates__4_29_2021__phenotypes__I83_I25_G56_N81_M16_M17_I48_C44_K80_I21__thresholds__6_7_8__max_samples__1000_10000_100000_300000_500000 /lab/corradin_biobank/FOR_AN/combinatorial_GWAS/.venv/bin/python /lab/corradin_biobank/FOR_AN/combinatorial_GWAS/notebooks/neural_network_training/interp.py -n dnn cnn -d 4_29_2021 -p I83 I25 G56 N81 M16 M17 I48 C44 K80 I21 -t 6 7 8 -m 1000 10000 100000 300000 500000


In [ ]:
# email = "premc@mit.edu"
# for dt in ['4_29_2021']:
#     for nt in ['dnn', 'cnn']:
#         for t in [6, 7, 8]:
#             for m in [1000, 10_000, 100_000, 300_000, 500_000]:
#                 cmd = interp(email, [nt], [dt], ['I83', 'I25', 'G56', 'N81', 'M16', 'M17', 'I48', 'C44', 'K80', 'I21'], [t], [m])
# #                 !{cmd}

In [ ]:
!bjobs -w -g /combigwas-interp

No job found in job group /combigwas-interp


In [ ]:
# !bkill -g /combigwas 0

Job <1529868> is being terminated
Job <1529867> is being terminated


In [ ]:
# 2 pairs
# I83 only
# DNN only
# None
# 1e-8, 1e-7, 1e-6
# 72
# X does not change between jobs
get_simulated_command = lambda email, pheno, threshold, max_samples, a1, a2:f"bsub -g /combigwas -q corradin -u '{email}' -J {pheno}_{threshold}_{max_samples} /lab/corradin_biobank/FOR_AN/combinatorial_GWAS/.venv/bin/python  /lab/corradin_biobank/FOR_AN/combinatorial_GWAS/notebooks/neural_network_training/simulated.py --phenotypes {pheno} --thresholds {threshold} --pair {a1} {a2} --create-test-output -m {max_samples} -r 42"

In [ ]:
!{get_simulated_command('premc@mit.edu', 'I83', 1e-8, 300000, *['6:134911816_G_A', '6:26118570_T_C'])}

Job <1526811> is submitted to queue <corradin>.


In [ ]:
!bpeek 1526811

Job <1526811> is not found


In [ ]:
# !bkill 1529871

Job <1529871> is being terminated


In [ ]:
lreg = lambda email, networks, dates, pheno, thresholds, max_samples: f"bsub -g /combigwas-interp -q corradin -u '{email}' -J interp_{get_name(networks, dates, pheno, thresholds, max_samples)} /lab/corradin_biobank/FOR_AN/combinatorial_GWAS/.venv/bin/python /lab/corradin_biobank/FOR_AN/combinatorial_GWAS/notebooks/neural_network_training/lr.py -n {join(networks, ' ')} -d {join(dates, ' ')} -p {join(pheno, ' ')} -t {join(thresholds, ' ')} -m {join(max_samples, ' ')}"

In [ ]:
!{lreg("premc@mit.edu", ['dnn'], ['4_29_2021'], ['I83'], [6], [1000, 10_000, 100_000, 300_000, 500_000])}

In [ ]:
!{lreg("pluthy@mit.edu", ['dnn'], ['4_29_2021'], ['I83'], [8], [1000])}

Job <1541767> is submitted to queue <corradin>.
